In [1]:
import pandas as pd
import numpy as np
import sys
import os
print(os.getcwd())
sys.path.append('/Users/calvindu/School/Projects/FINANCE_TRACKER/')

/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/dsci_src


In [2]:
from src.plugins.categorization_model.utils import validate_categories


category_labels = [
    'housing',
    'utilities',
    'telecommunication',
    'subscriptions',
    'food_drink',
    'alcohol',
    'groceries',
    'clothing_and_accessories',
    'personal_care',
    'health',
    'shopping',
    'furniture',
    'office_supplies',
    'education',
    'transportation',
    'travel_accomodations',
    'entertainment',
    'credit_card_payment',
    'debt_payments',
    'other',
    'income_wages',
    'income_reimbursement',
    'income_government',
    'income_investments',
    'income_other',
    'transfer_in',
    'transfer_out',
    'etransfer_in',
    'etransfer_out',
]

default_category_groupings = {
    'bills': ['housing', 'utilities', 'telecommunication'],
    'subscriptions': ['subscriptions'],
    'food_drink': ['food_drink', 'alcohol'],
    'groceries': ['groceries'],
    'clothing': ['clothing_and_accessories'],
    'health/wellbeing': ['health', 'personal_care'],
    'entertainment': ['entertainment'],
    'shopping': ['shopping', 'office_supplies', 'furniture'],
    'education': ['education'],
    'travel': ['travel_accomodations', 'transportation'],
    'money_transfer': ['credit_card_payment', 'transfer_in', 'transfer_out'],
    'debt_payments': ['debt_payments'],
    'other': ['other'],
    'salary': ['income_wages'],
    'other_income': ['income_government','income_investments','income_other'],
    'reimbursement': ['income_reimbursement'],
    'etransfer': ['etransfer_in', 'etransfer_out']
}

print(validate_categories(default_category_groupings, category_labels))

df = pd.read_csv('/Users/calvindu/School/categorized.csv')
df = df[['account', 'date', 'description', 'amount', 'category']]

print(df)

{'required_labels': [], 'extra_labels': []}


FileNotFoundError: [Errno 2] No such file or directory: '/Users/calvindu/School/categorized.csv'

In [ ]:
df['category'] = df['category'].fillna(0)
df = df[df['category'] != 0]

print(df)

          account        date                        description  amount  \
0    BMO_CHECKING  2023-07-13                    NEW HONG FATT B   28.00   
1    BMO_CHECKING  2023-07-13                    NEW HONG FATT B   28.00   
2    BMO_CHECKING  2023-07-12             TF 0005191230208404641  101.02   
3    BMO_CHECKING  2023-07-12             TF 0005191230208404641  101.02   
4            AMEX  2023-07-11          TIM HORTONS #2387 TORONTO   10.15   
..            ...         ...                                ...     ...   
649       CIBC_CC  2022-12-13  PAYMENT THANK YOU/PAIEMEN T MERCI -150.67   
650       CIBC_CC  2022-12-13      STARBUCKS 04574 VANCOUVER, BC    6.32   
651       CIBC_CC  2022-12-13  PAYMENT THANK YOU/PAIEMEN T MERCI   -6.32   
652       CIBC_CC  2022-12-13  PAYMENT THANK YOU/PAIEMEN T MERCI -148.84   
654       CIBC_CC  2022-12-12      MY HOME CUISINE VANCOUVER, BC   11.00   

                category  
0             food_drink  
1             food_drink  
2     

In [ ]:
from feature_engineering.transaction_matching_features import transaction_matching, get_matched_transactions, count_transactions_matching_feature
from feature_engineering.time_features import add_time_features
from feature_engineering.numerical_features import add_numerical_features
from feature_engineering.text_features import text_cleanup, add_text_features

In [ ]:
# Transaction Matching Features
df = transaction_matching(df=df, transaction_column='amount', date_column='date', time_window=3, new_matched_column='matched_transactions')
df = count_transactions_matching_feature(df=df)

# Time Features
df = add_time_features(df=df, date_column='date')

# Numerical Features
df = add_numerical_features(df=df, amount_column='amount')

# Text Features

df['cleaned_description'] = text_cleanup(df['description'])
df = add_text_features(df=df, desc_column='cleaned_description')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X = df.drop('category', axis=1)
y = df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define columns to be one-hot encoded and scaled
categorical_cols = ['account', 'amount_category', 'amount_sign'] 
numerical_cols = ['amount', 'matched_transactions_count', 'day_of_week', 'month_of_year', 'description_length', 'num_tokens']
removed_columns = ['date', 'matched_transactions', 'description', 'cleaned_description', 'category', 'pred_category', 'correct_category']

categorized_columns = categorical_cols + (numerical_cols)

numerical_cols.extend(set(list(df.columns)) - set(categorized_columns) - set(removed_columns))

# Create a ColumnTransformer to apply different preprocessing to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Add encoders
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Create a KNN classifier
k = 5  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the model
knn_classifier.fit(X_train_preprocessed, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test_preprocessed)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.75


In [ ]:
df = X_test
df['pred_category'] = y_pred
df['correct_category'] = y_test

df = df[['account', 'description', 'amount', 'pred_category', 'correct_category']]
print(df)

df.to_csv('/Users/calvindu/School/predicted_data.csv')

            account                           description  amount  \
497  BMO_MASTERCARD              ADIDAS CA 855-8234327 ON -140.00   
244  BMO_MASTERCARD            291 SPADINA AVE TORONTO ON   60.00   
552         CIBC_CC           FARM TO TABLE VANCOUVER, BC    9.26   
213  BMO_MASTERCARD            291 SPADINA AVE TORONTO ON   80.00   
549         CIBC_CC         STARBUCKS 04574 VANCOUVER, BC    2.54   
..              ...                                   ...     ...   
388    BMO_CHECKING                       NEW HONG FATT B   46.44   
322  BMO_MASTERCARD         EGG CLUB WELLESLEY TORONTO ON    9.71   
218    RBC_Chequing                        Email Trfs Can   20.00   
462  BMO_MASTERCARD  DOMINOS PIZZA #39032 604-733-2118 BC  100.14   
511         CIBC_CC                  CHOO TEA BURNABY, BC    8.06   

            pred_category      correct_category  
497  income_reimbursement  income_reimbursement  
244            food_drink            food_drink  
552            food_d

In [ ]:
import joblib


# Save the trained model and feature names using joblib
joblib.dump(knn_classifier, 'knn_model.joblib')

['knn_model.joblib']

In [ ]:
loaded_knn_model = joblib.load('knn_model.joblib')


# Now you can use 'loaded_knn_model' for making predictions
y_pred = loaded_knn_model.predict(X_test_preprocessed)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.75
